In [ ]:
import pymysql
import  math 

connection = pymysql.connect(host='localhost', user='auth_user', password='Aauth123', db='nba_stats')
cursor = connection.cursor()

In [ ]:
K = 32
INITIAL_ELO = 1500

def fetch_elo(team_id):
    cursor.execute("SELECT elo_rating FROM teams WHERE TEAM_ID = %s;", (team_id,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return INITIAL_ELO 

def update_elo_in_db(team_id, new_elo):
    cursor.execute("UPDATE teams SET elo_rating = %s WHERE TEAM_ID = %s;", (new_elo, team_id))
    connection.commit()

def update_elo_with_margin(old_rating, expected, actual, margin, max_K=32):
    # I wanted to consider the match points as I'd expect a better team to make more points
    factor = (max_K * 2) / (1 + math.exp(-margin / 12))  
    K = min(factor, max_K)
    return old_rating + K * (actual - expected)

My original database design didn't consider that I would need an ELO rating attribute... as the ELO rating is kept throughout the years then I consider it doesn''t need another table.

In [ ]:
try:
    cursor.execute("ALTER TABLE teams ADD elo_rating FLOAT DEFAULT 1500;")
    connection.commit()
except pymysql.err.InternalError:  
    print("Column elo_rating already exists, skipping alteration.")

##  Pull the data of each game

In [4]:
cursor.execute("SELECT GAME_ID FROM games ORDER BY GAME_DATE_EST;")
game_ids = [row[0] for row in cursor.fetchall()]

In [5]:
for game_id in game_ids:
    cursor.execute("SELECT TEAM_ID, PTS FROM team_stats WHERE GAME_ID = %s;", (game_id,))
    results = cursor.fetchall()
    
    team1_id, team1_pts = results[0]
    team2_id, team2_pts = results[1]

    team1_elo = fetch_elo(team1_id)
    team2_elo = fetch_elo(team2_id)

    expected_team1 = 1 / (1 + 10 ** ((team2_elo - team1_elo) / 400))
    actual_team1 = 1 if team1_pts > team2_pts else 0

    margin = abs(team1_pts - team2_pts)
    new_team1_elo = update_elo_with_margin(team1_elo, expected_team1, actual_team1, margin)
    new_team2_elo = update_elo_with_margin(team2_elo, 1 - expected_team1, 1 - actual_team1, margin)
    
    update_elo_in_db(team1_id, new_team1_elo)
    update_elo_in_db(team2_id, new_team2_elo)

cursor.close()
connection.close()